## Innomatics Research Labs – Advanced GenAI Internship Entrance Test

**Objective:**  
To integrate multiple real-world data sources (CSV, JSON, SQL), create a unified dataset, and perform analytical queries to simulating real-world systems.

**Dataset Files Used:**
- orders.csv (Transactional Data)
- users.json (User Master Data)
- restaurants.sql (Restaurant Master Data)

**Final Output:**
- final_food_delivery_dataset.csv


In [3]:
import pandas as pd
import numpy as np
import sqlite3

## Step 1: Load Orders Data (CSV)

This dataset contains transactional order information

In [14]:
orders = pd.read_csv("orders.csv")
orders['order_date'] = pd.to_datetime(orders['order_date'])

orders.head()


C:\Users\Namrata\AppData\Local\Temp\ipykernel_13172\1279267129.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  orders['order_date'] = pd.to_datetime(orders['order_date'])


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,2023-02-18,842.97,New Foods Chinese
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,2023-07-15,163.93,Spice Kitchen Punjabi
3,4,319,224,2023-10-04,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,2023-12-25,1321.91,Royal Eatery South Indian


## Step 2: Load Users Data (JSON)

This dataset contains user master information 

In [9]:
users = pd.read_json("users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


## Step 3: Load Restaurants Data (SQL)

The restaurant master data is provided as an SQL script.
We load it into an in-memory SQLite database and read it as a DataFrame.


In [10]:
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

with open("restaurants.sql", "r") as file:
    cursor.executescript(file.read())

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


## Step 4: Merge Datasets

We perform LEFT JOIN operations to retain all order records.

Join Keys:
- orders.user_id → users.user_id
- orders.restaurant_id → restaurants.restaurant_id

In [12]:
df = orders.merge(users, on="user_id", how="left")
df = df.merge(restaurants, on="restaurant_id", how="left")

df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,2023-02-18,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,2023-07-15,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,2023-10-04,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,2023-12-25,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


## Step 5: Save Final Dataset
The final dataset contains
- Order details
- User information
- Restaurant information

In [34]:
df.to_csv("final_food_delivery_dataset.csv", index=False)


In [35]:
pd.read_csv("final_food_delivery_dataset.csv").columns


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating', 'rating_range', 'quarter'],
      dtype='str')

## Multiple Choice Questions
This section contains MCQs based on the given datasets. Analyze the data carefully and select the correct answers.


**Q1. Which city has the highest total revenue from Gold members?**

In [16]:
gold_df = df[df['membership'] == 'Gold']

gold_df.groupby('city')['total_amount'].sum().sort_values(ascending=False)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

**Q2. Which cuisine has the highest average order value across all orders?**

In [17]:
df.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

**Q3. How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?**

In [19]:
user_spend = df.groupby('user_id')['total_amount'].sum()
(user_spend > 1000).sum()

np.int64(2544)

**Q4. Which restaurant rating range generated the highest total revenue?**

In [21]:
df['rating_range'] = pd.cut(
    df['rating'],
    bins=[3.0, 3.5, 4.0, 4.5, 5.0]
)

df.groupby('rating_range')['total_amount'].sum().sort_values(ascending=False)


rating_range
(4.5, 5.0]    2197030.75
(4.0, 4.5]    1960326.26
(3.0, 3.5]    1881754.57
(3.5, 4.0]    1717494.41
Name: total_amount, dtype: float64

**Q5. Among Gold members, which city has the highest average order value?**

In [22]:
gold_df.groupby('city')['total_amount'].mean().sort_values(ascending=False)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

**Q6. Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?**

In [23]:
cuisine_analysis = df.groupby('cuisine').agg(
    restaurant_count=('restaurant_id', 'nunique'),
    total_revenue=('total_amount', 'sum')
)

cuisine_analysis.sort_values(
    ['restaurant_count', 'total_revenue'],
    ascending=[True, False]
)


,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


**Q7. What percentage of total orders were placed by Gold members? (Rounded to nearest integer)**

In [24]:
gold_orders = df[df['membership'] == 'Gold'].shape[0]
total_orders = df.shape[0]

round((gold_orders / total_orders) * 100)


50

**Q8. Which restaurant has the highest average order value but less than 20 total orders?**

In [37]:
options = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]

filtered = restaurant_stats.loc[
    restaurant_stats.index.isin(options)
]

filtered[filtered['total_orders'] < 20] \
    .sort_values('average_order_value', ascending=False)


,average_order_value,total_orders
restaurant_name,,
Ruchi Foods Chinese,686.603158,19


**Q9. Which combination contributes the highest revenue?**

In [26]:
df.groupby(['membership', 'cuisine'])['total_amount'] \
  .sum() \
  .sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

**Q10. During which quarter of the year is the total revenue highest?**

In [27]:
df['quarter'] = df['order_date'].dt.to_period('Q')

df.groupby('quarter')['total_amount'] \
  .sum() \
  .sort_values(ascending=False)


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

### Numerical Answers

**Q1. How many total orders were placed by users with Gold membership?**

In [38]:
gold_orders_count = df[df['membership'] == 'Gold'].shape[0]
gold_orders_count


4987

**Q2. What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?**


In [39]:
hyderabad_revenue = df[df['city'] == 'Hyderabad']['total_amount'].sum()
round(hyderabad_revenue)


1889367

**Q3. How many distinct users placed at least one order?**


In [40]:
df['user_id'].nunique()


2883

**Q4. What is the average order value (rounded to 2 decimals) for Gold members?**


In [41]:
gold_aov = df[df['membership'] == 'Gold']['total_amount'].mean()
round(gold_aov, 2)


np.float64(797.15)

**Q5. How many orders were placed for restaurants with rating ≥ 4.5?**

In [42]:
high_rating_orders = df[df['rating'] >= 4.5].shape[0]
high_rating_orders


3374

**Q6. How many orders were placed in the top revenue city among Gold members only?**

In [44]:
top_gold_city = (
    df[df['membership'] == 'Gold']
    .groupby('city')['total_amount']
    .sum()
    .idxmax()
)

gold_orders_top_city = df[
    (df['membership'] == 'Gold') &
    (df['city'] == top_gold_city)
].shape[0]

gold_orders_top_city


1337

In [45]:
df.shape[0]


10000